In [6]:
import sqlite3
from contextlib import closing

In [8]:
with closing(sqlite3.connect("../db/simulations.db")) as connection:
    with closing(connection.cursor()) as cursor:
        rows = cursor.execute("SELECT * from results").fetchall()
        print(rows)

[(0.492291161, 1, 1, 3, 2, 3, 2, 'inmates_enriched_10k', 'qualitative', 0.4216496970595123, 0.13757713376398029, 0.1, 1.0), (0.229804543, 1, 2, 3, 2, 3, 2, 'inmates_enriched_10k', 'qualitative', 0.4216496970595123, 0.13757713376398029, 0.1, 1.0), (0.460659098, 1, 1, 3, 3, 3, 2, 'inmates_enriched_10k', 'qualitative', 0.47868924815734026, 0.08782206187217996, 0.1, 1.0), (0.464771934, 1, 2, 3, 3, 3, 2, 'inmates_enriched_10k', 'qualitative', 0.47868924815734026, 0.08782206187217996, 0.1, 1.0), (0.454780562, 1, 3, 3, 3, 3, 2, 'inmates_enriched_10k', 'qualitative', 0.47868924815734026, 0.08782206187217996, 0.1, 1.0), (0.347857692, 1, 1, 3, 2, 3, 3, 'inmates_enriched_10k', 'qualitative', 0.4289241448782952, 0.12611941035202437, 0.1, 1.0), (0.481055865, 1, 2, 3, 2, 3, 3, 'inmates_enriched_10k', 'qualitative', 0.4289241448782952, 0.12611941035202437, 0.1, 1.0), (0.497336361, 1, 1, 3, 3, 3, 3, 'inmates_enriched_10k', 'qualitative', 0.7230307641732675, 0.0050003168402566, 0.1, 1.0), (0.851508632,